In [1]:
import pyspark.sql.functions as fn

In [2]:
# set a spark context
sc = spark.sparkContext

# define our data as JSON
people_json = ['[{"name":{"first":"Alice", "last": "Summers"},"age": "34", "cars": ["honda"],  "locations":[{"type": "work", "city":"Glasgow"}, {"type": "home", "city": "Edinburgh"}]},\
                 {"name":{"first":"Bob", "last": "Winters"},"age": "33", "cars": ["ford", "BMW"], "locations":[{"type": "work", "city":"Glasgow"}, {"type": "home", "city": "Edinburgh"}]},\
                 {"name":{"first":"Charlie", "last": "Spring"},"age": "35", "cars": ["vauxhall", "peugeot", "VW"], "locations":[{"type": "work", "city":"Bristol"}, {"type": "home", "city": "Bath"}, {"type":"holiday", "city": "Paris"}]}]']

# convert to RDD
people_rdd = sc.parallelize(people_json)
# convert to dataframe
people_df = spark.read.json(people_rdd)
# rearrange column order
people_df = people_df.select("age", "name", "cars", "locations")

display(people_df)

age,name,cars,locations
34,"List(Alice, Summers)",List(honda),"List(List(Glasgow, work), List(Edinburgh, home))"
33,"List(Bob, Winters)","List(ford, BMW)","List(List(Glasgow, work), List(Edinburgh, home))"
35,"List(Charlie, Spring)","List(vauxhall, peugeot, VW)","List(List(Bristol, work), List(Bath, home), List(Paris, holiday))"


In [3]:
people_df.printSchema()

root
-- age: string (nullable = true)
-- name: struct (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
-- cars: array (nullable = true)
 |-- element: string (containsNull = true)
-- locations: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- city: string (nullable = true)
 | |-- type: string (nullable = true)

In [4]:
# extracting data from a map (dictionary) column
surname_df = people_df.withColumn("surname", fn.col("name")["last"])
display(surname_df)

age,name,cars,locations,surname
34,"List(Alice, Summers)",List(honda),"List(List(Glasgow, work), List(Edinburgh, home))",Summers
33,"List(Bob, Winters)","List(ford, BMW)","List(List(Glasgow, work), List(Edinburgh, home))",Winters
35,"List(Charlie, Spring)","List(vauxhall, peugeot, VW)","List(List(Bristol, work), List(Bath, home), List(Paris, holiday))",Spring


In [5]:
# extracting data from an array (list) columnn
first_car = people_df.withColumn("first_car", fn.col("cars")[0])
display(first_car)

age,name,cars,locations,first_car
34,"List(Alice, Summers)",List(honda),"List(List(Glasgow, work), List(Edinburgh, home))",honda
33,"List(Bob, Winters)","List(ford, BMW)","List(List(Glasgow, work), List(Edinburgh, home))",ford
35,"List(Charlie, Spring)","List(vauxhall, peugeot, VW)","List(List(Bristol, work), List(Bath, home), List(Paris, holiday))",vauxhall


In [6]:
# extracting a defined value to a new dataframe
single_value_people_df = people_df.withColumn("first_city", fn.col("locations")[0]["city"])
display(single_value_people_df)

age,name,cars,locations,first_city
34,"List(Alice, Summers)",List(honda),"List(List(Glasgow, work), List(Edinburgh, home))",Glasgow
33,"List(Bob, Winters)","List(ford, BMW)","List(List(Glasgow, work), List(Edinburgh, home))",Glasgow
35,"List(Charlie, Spring)","List(vauxhall, peugeot, VW)","List(List(Bristol, work), List(Bath, home), List(Paris, holiday))",Bristol


In [7]:
expanded_people_df = people_df.withColumn("cities", fn.col("locations")["city"])

In [8]:
display(expanded_people_df)

age,name,cars,locations,cities
34,"List(Alice, Summers)",List(honda),"List(List(Glasgow, work), List(Edinburgh, home))","List(Glasgow, Edinburgh)"
33,"List(Bob, Winters)","List(ford, BMW)","List(List(Glasgow, work), List(Edinburgh, home))","List(Glasgow, Edinburgh)"
35,"List(Charlie, Spring)","List(vauxhall, peugeot, VW)","List(List(Bristol, work), List(Bath, home), List(Paris, holiday))","List(Bristol, Bath, Paris)"


In [9]:
extracted_people_df = people_df.withColumn("location", fn.col("locations")[0])

In [10]:
display(extracted_people_df)

age,name,cars,locations,location
34,"List(Alice, Summers)",List(honda),"List(List(Glasgow, work), List(Edinburgh, home))","List(Glasgow, work)"
33,"List(Bob, Winters)","List(ford, BMW)","List(List(Glasgow, work), List(Edinburgh, home))","List(Glasgow, work)"
35,"List(Charlie, Spring)","List(vauxhall, peugeot, VW)","List(List(Bristol, work), List(Bath, home), List(Paris, holiday))","List(Bristol, work)"


In [11]:
# Alternatively, a DataFrame can be created for a JSON dataset represented by
# an RDD[String] storing one JSON object per string
class_json = ['[{"subject":"maths", "performance":{"names": ["Alice", "Bob", "Charlie", "Daniel", "Emily"], "scores":[45,56,67,34,89], "grades": ["D", "C", "B", "E", "A"]}},\
                 {"subject":"english", "performance":{"names": ["Alice", "Bob", "Charlie", "Daniel", "Emily"], "scores":[79,54,62,39,64], "grades": ["A", "C", "B", "E", "B"]}},\
                 {"subject":"history", "performance":{"names": ["Alice", "Bob", "Charlie", "Daniel", "Emily"], "scores":[47,76,61,44,79], "grades": ["D", "A", "B", "D", "A"]}}]']
class_rdd = sc.parallelize(class_json)
class_df = spark.read.json(class_rdd)
class_df = class_df.select("subject", "performance")
class_df.show()

+-------+--------------------+
subject| performance|
+-------+--------------------+
 maths|[[D, C, B, E, A],...|
english|[[A, C, B, E, B],...|
history|[[D, A, B, D, A],...|
+-------+--------------------+

In [12]:
display(class_df)

subject,performance
maths,"List(List(D, C, B, E, A), List(Alice, Bob, Charlie, Daniel, Emily), List(45, 56, 67, 34, 89))"
english,"List(List(A, C, B, E, B), List(Alice, Bob, Charlie, Daniel, Emily), List(79, 54, 62, 39, 64))"
history,"List(List(D, A, B, D, A), List(Alice, Bob, Charlie, Daniel, Emily), List(47, 76, 61, 44, 79))"


In [13]:
# we can pull out a specified value from a map of arrays to a new column
grade_df = class_df.withColumn("first_grade", fn.col("performance")["grades"][0])

In [14]:
display(grade_df)

subject,performance,first_student_grade
maths,"List(List(D, C, B, E, A), List(Alice, Bob, Charlie, Daniel, Emily), List(45, 56, 67, 34, 89))",D
english,"List(List(A, C, B, E, B), List(Alice, Bob, Charlie, Daniel, Emily), List(79, 54, 62, 39, 64))",A
history,"List(List(D, A, B, D, A), List(Alice, Bob, Charlie, Daniel, Emily), List(47, 76, 61, 44, 79))",D


In [15]:
# however we cannot directly extract all first elements from a map of arrays
extracted_class_df = class_df.withColumn("first_student", fn.col("performance")[0])
# this gives an ERROR!

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o914.withColumn.
: org.apache.spark.sql.AnalysisException: Field name should be String Literal, but it's 0;
	at org.apache.spark.sql.catalyst.expressions.ExtractValue$.apply(complexTypeExtractors.scala:73)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve(Analyzer.scala:914)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve$2.apply(Analyzer.scala:915)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve$2.apply(Analyzer.scala:915)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveReferences$$resolve(Analyzer.scala:915)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9$$anonfun$applyOrElse$36.apply(Analyzer.scala:993)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9$$anonfun$applyOrElse$36.apply(Analyzer.scala:993)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9.applyOrElse(Analyzer.scala:993)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9.applyOrElse(Analyzer.scala:918)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsU

In [16]:
# using .* to extract a struct column to separate columns for onward processing
separated_class_df = class_df.select("subject", "performance.*")
    
display(separated_class_df)    

subject,grades,names,scores
maths,"List(D, C, B, E, A)","List(Alice, Bob, Charlie, Daniel, Emily)","List(45, 56, 67, 34, 89)"
english,"List(A, C, B, E, B)","List(Alice, Bob, Charlie, Daniel, Emily)","List(79, 54, 62, 39, 64)"
history,"List(D, A, B, D, A)","List(Alice, Bob, Charlie, Daniel, Emily)","List(47, 76, 61, 44, 79)"
